In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pdb
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import numpy as np
import torch

# For visualizer
import rospy
from Data.utils import *
from visualization_msgs.msg import *
from Models.DiscreteBKI import *
rospy.init_node('talker',disable_signals=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device == "cuda":
  start = torch.cuda.Event(enable_timing=True)
  end = torch.cuda.Event(enable_timing=True)
else:
  start = None
  end = None
print("device is ", device)
    
home_dir = os.path.expanduser('~')
dataset_loc = os.path.join(home_dir, "Data/Rellis-3D/sequences")

device is  cuda


In [3]:
bki_map = DiscreteBKI(
    torch.tensor([256, 256, 16]).to(device), # Grid size
    torch.tensor([-25.6, -25.6, -2.0]).to(device), # Lower bound
    torch.tensor([25.6, 25.6, 1.2]).to(device), # Upper bound
    device=device
)

# Add visualization
map_pub = rospy.Publisher('SemMap', MarkerArray, queue_size=10)

In [ ]:
# Load point cloud from RELLIS
velo_loc = os.path.join(dataset_loc, "00004", "os1_cloud_node_kitti_bin")
label_base_loc = os.path.join(dataset_loc, "00004", "os1_cloud_node_semantickitti_label_id")
os_files = os.listdir(velo_loc)

current_map = bki_map.initialize_grid()

curr_frame_id=0
end_frame_id=0
for velo_file in sorted(os_files):
    velo = np.fromfile(os.path.join(velo_loc, velo_file), dtype=np.float32).reshape(-1, 4)[:, :3]
    velo = torch.from_numpy(velo).to(device)
    labels = np.fromfile(os.path.join(label_base_loc, velo_file.split(".")[0]+".label"), dtype=np.uint32)
    labels_remapped = torch.from_numpy(LABELS_REMAP[labels]).to(device=device) # Remap labels to be contiguous
    
    # Ego vehicle = 0
    non_void = labels_remapped != 0
    velo = velo[non_void]
    labels_remapped = labels_remapped[non_void]
    
    labeled_pc = torch.hstack( (velo, labels_remapped.reshape(-1, 1)) )
    
    non_dynamic = (labels_remapped != LABELS_REMAP[8]) & (labels_remapped != LABELS_REMAP[17])
    labeled_pc = labeled_pc[non_dynamic]
    
    posterior_map = bki_map(current_map, labeled_pc)

    publish_voxels(posterior_map, map_pub, 
        bki_map.centroids, 
        bki_map.min_bound.reshape(-1), 
        bki_map.max_bound.reshape(-1), 
        bki_map.grid_size.reshape(-1)
    )

    if curr_frame_id==end_frame_id:
        break
    curr_frame_id += 1

In [ ]:

print(torch.unique(torch.argmax(posterior_map, dim=-1), return_counts=True))

In [ ]:
# Test 3D conv

num_classes = 20

# X, Y, Z
filters = torch.zeros(27, dtype=torch.float)
filters[13] = 1
filters = filters.view(1, 1, 3, 3, 3)

print(filters[0, 0, 1, :, :])

inputs = torch.ones(num_classes, 1, 5, 5, 5)

output = F.conv3d(inputs, filters, padding="same")
print(output[0, 0, :, :, 0])

In [ ]:
# pred_dir = os.path.join(dataset_loc, "00004", "os1_cloud_node_semantickitti_label_id", "000000.label")
# pred_file = np.fromfile(pred_dir, dtype=np.uint32).reshape((-1))

# pred_labels = pred_file & 0xFFFF
# pdb.set_trace()
# print(pred_file.shape)

In [ ]:
import pdb
import time
import torch

from torch.utils.data import Dataset, DataLoader
from Data.dataset import Rellis3dDataset

train_dir = dataset_loc

rellis_ds = Rellis3dDataset(directory=train_dir, device=device, num_frames=20, remap=True)
dataloader_train = DataLoader(rellis_ds, batch_size=1, shuffle=False, collate_fn=rellis_ds.collate_fn, num_workers=2)

idx = 0
current_map = bki_map.initialize_grid()
curr_time = time.time()

# Test running bki map with initial filter on
for current_points, current_labels, current_voxels in dataloader_train:
    batch_labeled_pc = torch.zeros(size=(0, 4), device=device)
    for b in range(len(current_points)):
        for f in range(len((current_points[b]))):
            pc = torch.from_numpy(current_points[b][f]).to(device)
            labels = torch.from_numpy(current_labels[b][f].astype(np.int32)).to(device).reshape(-1, 1)
            labeled_pc = torch.hstack( (pc, labels))
            batch_labeled_pc = torch.vstack((batch_labeled_pc, labeled_pc))

    # Publish each point cloud to rviz
    # pdb.set_trace()
    print("Elapsed time for collating ", time.time() - curr_time)
    curr_time = time.time()
    posterior_map = bki_map(current_map, batch_labeled_pc)
    print("Elapsed time for forward pass ", time.time() - curr_time)
    curr_time = time.time()

    if idx%20==0:
        publish_voxels(posterior_map, map_pub, 
            bki_map.centroids,
            bki_map.min_bound.reshape(-1),
            bki_map.max_bound.reshape(-1),
            bki_map.grid_size.reshape(-1))
        print("Elapsed publish map time ", time.time() - curr_time)

    idx += 1
    curr_time = time.time()


In [7]:
import os
import pdb
import json
import time
import torch
import torch.optim as optim
from model_utils import *
from torch import nn
from torch.utils.data import Dataset, DataLoader
from Data.dataset import Rellis3dDataset
from torch.utils.tensorboard import SummaryWriter

#CONSTANTS
SEED = 42
NUM_CLASSES = colors.shape[0]
TRAIN_DIR = dataset_loc
NUM_FRAMES = 20
MODEL_NAME = "DiscreteBKI"
NUM_WORKERS = 4
EPOCH_NUM = 500

#Model Parameters
class_frequencies = CLASS_COUNTS_REMAPPED
epsilon_w = 0.001  # eps to avoid zero division
weights = torch.from_numpy(1 / np.log(class_frequencies + epsilon_w)).to(torch.float32)
criterion = nn.CrossEntropyLoss(weight=weights.to(device))

model_params_file = os.path.join(TRAIN_DIR, sorted(os.listdir(TRAIN_DIR))[0], 'params.json')
with open(model_params_file) as f:
    grid_params = json.load(f)
    grid_params['grid_size'] = [ int(p) for p in grid_params['grid_size'] ]

# Load model
lr = 0.00001
BETA1 = 0.9
BETA2 = 0.999
model, B, decayRate = get_model(MODEL_NAME, grid_params=grid_params, device=device)

rellis_ds = Rellis3dDataset(directory=TRAIN_DIR, device=device, num_frames=NUM_FRAMES, remap=True)
dataloader = DataLoader(rellis_ds, batch_size=B, shuffle=False, collate_fn=rellis_ds.collate_fn, num_workers=NUM_WORKERS)

model_name = MODEL_NAME + "_" + str(NUM_CLASSES)

writer = SummaryWriter("./Models/Runs/" + model_name)
save_dir = "./Models/Weights/" + model_name

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# Optimizer setup
setup_seed(SEED)
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(BETA1, BETA2))
my_lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
    T_max=200, eta_min=1e-6)

#torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

train_count = 0
current_map = bki_map.initialize_grid()
for epoch in range(EPOCH_NUM):
    # Training
    model.train()
    for points, points_labels, voxels, invalid_voxels in dataloader:
        optimizer.zero_grad()
        
        frame_labeled_pc = torch.zeros(size=(0, 4), device=device, requires_grad=True)
        voxels_labels= torch.tensor(
            voxels, dtype=torch.long
        ).to(device).squeeze()
        valid_voxels_mask = torch.logical_not(torch.tensor(
            invalid_voxels, dtype=torch.bool
        ).to(device).squeeze())

        # Assumes batch size=1 TODO: change later
        for f in range(len(points[0])):
            pc = torch.from_numpy(points[0][f]).to(device)
            labels = torch.from_numpy(points_labels[0][f].astype(np.int32)).to(device).reshape(-1, 1)

            labeled_pc = torch.hstack( (pc, labels))
            frame_labeled_pc = torch.vstack((frame_labeled_pc, labeled_pc))

        # TODO: modify bkimap forward pass to be batched
        preds = bki_map(current_map, frame_labeled_pc)
    
        # Exclude free space, invalid voxels, and nonupdated map cells
        free_space_mask = voxels_labels > 0
        prior_mask = torch.logical_not(torch.all(preds==bki_map.prior, dim=-1))
        valid_voxels_mask = free_space_mask & valid_voxels_mask & prior_mask
        voxels_labels = voxels_labels[valid_voxels_mask]
        preds_masked = preds[valid_voxels_mask]

        # Debugging mask 
        # publish_voxels(preds, map_pub, 
        #     bki_map.centroids,
        #     bki_map.min_bound.reshape(-1),
        #     bki_map.max_bound.reshape(-1),
        #     bki_map.grid_size.reshape(-1),
        #     valid_voxels_mask=valid_voxels_mask.flatten())
        # pdb.set_trace()
        # publish_pc(
        #     frame_labeled_pc[:, :3], frame_labeled_pc[:, 3].detach(), map_pub,
        #     bki_map.min_bound.reshape(-1),
        #     bki_map.max_bound.reshape(-1),
        #     bki_map.grid_size.reshape(-1)
        # )
        # pdb.set_trace()

        voxels_labels       = voxels_labels.view(-1).long()
        preds               = preds.view(-1).long()
        loss = criterion(preds_masked, voxels_labels)
        # pdb.set_trace()
        loss.backward()
        optimizer.step()

        # Accuracy
        with torch.no_grad():
            probs = nn.functional.softmax(preds_masked, dim=1)
            preds_masked = np.argmax(probs.detach().cpu().numpy(), axis=1)
            voxels_np = voxels_labels.detach().cpu().numpy()
            accuracy = np.sum(preds_masked == voxels_np) / voxels_np.shape[0]

        # Record
        writer.add_scalar(MODEL_NAME + '/Loss/Train', loss.item(), train_count)
        writer.add_scalar(MODEL_NAME + '/Accuracy/Train', accuracy, train_count)
            
        train_count += len(points)
            
    # Save model, decreaser learning rate
    my_lr_scheduler.step()
    torch.save(model.state_dict(), os.path.join(save_dir, "Epoch" + str(epoch) + ".pt"))
        

> /tmp/ipykernel_321737/4124371507.py(103)<cell line: 63>()
    101             valid_voxels_mask=valid_voxels_mask.flatten())
    102         pdb.set_trace()
--> 103         publish_pc(
    104             frame_labeled_pc[:, :3], frame_labeled_pc[:, 3].detach(), map_pub,
    105             bki_map.min_bound.reshape(-1),

> /tmp/ipykernel_321737/4124371507.py(111)<cell line: 63>()
    109         pdb.set_trace()
    110 
--> 111         voxels_labels       = voxels_labels.view(-1).long()
    112         preds               = preds.view(-1).long()
    113         loss = criterion(preds_masked, voxels_labels)

> /tmp/ipykernel_321737/4124371507.py(102)<cell line: 63>()
    100             bki_map.grid_size.reshape(-1),
    101             valid_voxels_mask=valid_voxels_mask.flatten())
--> 102         pdb.set_trace()
    103         publish_pc(
    104             frame_labeled_pc[:, :3], frame_labeled_pc[:, 3].detach(), map_pub,

> /tmp/ipykernel_321737/4124371507.py(111)<cell line: 63

BdbQuit: 